This is for viewing stats and figuring out problems (e.g., who hasn't done a review, etc)


# todo 

way of telling who has turned in essay but has no reviewer

way of telling who turned in essay etc after the final close of metareview window

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Dynamically sets location so do not have to manually toggle between dropbox and documents
import os
path = "/".join([a for a in os.path.abspath("").split('/') if a not in ['Notebooks', 'personal']])
%cd $path

import pandas as pd
pd.options.display.max_rows = 999
import os

#Plotting 
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

sns.set_palette(sns.color_palette('plasma'))


from CanvasHacks import environment
from CanvasHacks.Configuration import InteractiveConfiguration
# from CanvasHacks.PeerReviewed.Definitions import TopicalAssignment, ContentAssignment
from CanvasHacks.Models.QuizModels import QuizData

# Repos
from CanvasHacks.Repositories.DataManagement import DataStore
from CanvasHacks.Repositories.factories import WorkRepositoryFactory, WorkRepositoryLoaderFactory
from CanvasHacks.Repositories.quizzes import QuizRepository, ReviewRepository
from CanvasHacks.Repositories.codes import AccessCodeRepo
from CanvasHacks.Repositories.reviewer_associations import assign_reviewers, AssociationRepository
from CanvasHacks.Repositories.students import StudentRepository
from CanvasHacks.Repositories.submissions import SubmissionRepository, QuizSubmissionRepository
# from CanvasHacks.Repositories.quizzes import process_work, load_student_work, remove_non_final_attempts, make_drop_list, save_json, drop_columns_from_frame

from CanvasHacks.Displays.dashboard import ControlStore

# Filesystem
from CanvasHacks.Files.FileTools import makeDataFileIterator, create_folder
from CanvasHacks.Files.QuizReportFileTools import load_new
# from CanvasHacks.QuizReportFileTools import  make_quiz_repo, load_activity_data_from_files

# Widgets
from CanvasHacks.Widgets.AssignmentSelection import make_assignment_chooser, view_selected_assignments, view_ungraded_assignments
from CanvasHacks.Widgets.LiveSelection import make_test_selector
from CanvasHacks.Widgets.AssignmentSelection import make_unit_chooser, make_selection_button
from CanvasHacks.Widgets.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.Widgets.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button


In [ ]:
units = [1,2,3,4, 5,6, 7 ]

# units = [i for i in range(1,8)]
# units.append(8)

control_store = ControlStore()
[control_store.load_unit(i) for i in units]

# Plots

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')

from bokeh.plotting import figure, show, output_notebook
from bokeh.transform import dodge
from bokeh.models import ColumnDataSource, Band, Span, Plot, HoverTool
# output to static HTML 
output_notebook()

try:
    j = [control_store._get_unit_run_data(u) for u in units]
    j = [k for k in j if k is not None]
except AttributeError as e:
    print(e)

d = pd.DataFrame(j)
# d = pd.DataFrame([control_store._get_unit_run_data(u) for u in units] )

renames = {
    'unit_number' : 'Unit',
    'essay' : 'Essay',
    'no_essay' : "No Essay",
    'skaa_review' : 'Review',
    'skaa_no_review' : "No Review",
    'skaa_metareview' : "Metareview",
    'skaa_no_metareview' : 'No Metareview'
}

d.rename(renames, axis=1, inplace=True)
# d

def plot_summary(frame, complete_name, incomplete_name):

    ESSAY_TOOLTIPS = [
                ("Essays submitted", "@Essay{0,0}"),
    #         ("date", "@date")
    #     ("avg total", "@total_patients{0,0}"),
    #         ("date", "@date")
    ]
    
    TOOLTIPS = ESSAY_TOOLTIPS if complete_name == 'Essay' else [] 

    p = figure(
        plot_width=1000,
        plot_height=400,
        title=f"{complete_name} ",
    #     x_axis_type='datetime',
               tooltips=TOOLTIPS
        
    )

    #     bar_data = frame.reset_index()
    #     if trim_pos is not None:
    #         bar_data = bar_data[bar_data[field_name] <= trim_pos]
    #     if trim_neg is not None:
    #         bar_data = bar_data[bar_data[field_name] >= trim_neg]

    source = ColumnDataSource(frame)
    p.xaxis.axis_label = "Unit"
    p.yaxis.axis_label = "# Students"
    
    # add a circle renderer with a size, color, and alpha
    # p.circle(x='date', y='new_confirmed_cases', size=10, color="navy", source=source, alpha=0.5)
    p.vbar(
        x='Unit',
        top=complete_name,
        color="navy",
        width=0.3,
        source=source,
        alpha=0.8,
        legend_label=complete_name)

    p.vbar(
#         x='Unit',
            x=dodge('Unit', -0.3, range=p.x_range),
        top=incomplete_name,
        color="firebrick",
        width=0.3,
        source=source,
        alpha=0.8,
        legend_label=incomplete_name,
)


# source = ColumnDataSource(data=dict(x=x, counts=counts))
    
    # p.line(
    #     x='date',
    #     y='total_patients',
    #     line_width=5,
    #     color="navy",
    #     source=source2,
    #     alpha=0.3,
    #     legend_label='7-day rolling avg')

    p.legend.location = 'top_right'

    show(p)

plot_summary(d, 'Essay', 'No Essay')

plot_summary(d, 'Review', 'No Review')

plot_summary(d, 'Metareview', 'No Metareview')

In [ ]:
d

# When things get submitted

In [ ]:
from bokeh.transform import jitter
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6, Plasma, mpl

# MAX_UNIT = 8

# units = [u for u in range(1,MAX_UNIT + 1)]
subs = []

for u in units:
    try:
        for i, r in control_store.skaa_repos[u].data.iterrows():
            subs.append({ 'unit' : u , 'submitted' : r.invited_to_review} )
    except (KeyError, AttributeError):
        pass

subs = pd.DataFrame(subs)
subs.dropna(inplace=True)
source = ColumnDataSource(subs)

mapper = linear_cmap(field_name='unit', palette=mpl['Plasma'][7] ,low=min(units) ,high=max(units))

p = figure(
        plot_width=1000,
        plot_height=500,
        title=f"Essay submissions: Number submitted by day",
        x_axis_type='datetime') #,
p.yaxis.axis_label = "Unit"
p.circle(x='submitted', 
         y=jitter('unit', width=0.6, range=p.y_range),
         legend_label='unit', 
         color=mapper, 
         source=source, 
         alpha=0.9)
p.legend.location = 'top_left'

show(p)

# sources = []
sources = {}

for u in units:
    repo = control_store.skaa_repos[u]
    if len(repo.data[repo.data.invited_to_review.notna()]) > 0:
        c = repo.data.invited_to_review.copy(deep=True)
        c = pd.DataFrame(c).dropna().rename({'invited_to_review' : 'date'}, axis=1).set_index('date')
        c['cum_subs'] = 1
        c = c.resample('1D').count()
        c['cum_subs'] = c.cum_subs.cumsum()
        c['unit'] = u
        sources[u] = ColumnDataSource(c.reset_index())

#     sources.append((u, ColumnDataSource(c.reset_index())))
#     sources.append(c)

# sources = pd.concat(sources)

def make_color_iter(number=8):
    for m in mpl['Plasma'][number]:
        yield m
    

enrollment = len(control_store.skaa_repos[1].studentRepo.data)

ci = make_color_iter()

TOOLTIPS = [
    ("Total submissions", "@cum_subs{0,0}"),
        ("Unit", "@unit"),
#     ("Date", "@date")
]


p = figure(
        plot_width=1000,
        plot_height=500,
        title=f"Essay submissions: Cumulative submissions",
        x_axis_type='datetime',
tooltips=TOOLTIPS) #,

p.yaxis.axis_label = "Cumulative submissions"


for unit, s in sources.items():
    p.line(
    x='date',
    y='cum_subs',
    line_width=5,
    color=next(ci),
    source=s,
    alpha=0.8,
    legend_label=f"Unit {unit}"
    )

enrolled = Span(location=enrollment,
                dimension='width', line_color='red',
                line_dash='dashed', line_width=3)
p.add_layout(enrolled)
    
p.legend.location = 'bottom_left'

show(p)

In [ ]:
subs[subs.submitted > '2023-12-15']

# Which students have / have not done stuff

In [ ]:
make_test_selector()
make_unit_chooser(num_units=8)


In [ ]:
# Added new control store so don't have to wait for all units to download 
# by using the instance above
control_store = ControlStore()
control_store.load_unit(environment.CONFIG.unit.unit_number)

repo = control_store.skaa_repos[environment.CONFIG.unit.unit_number]

select * from review_associations ra left join invitation_received ir 
on ra.assessor_id = ir.student_id  where ir.sent_at is NULL;

select * from review_associations ra left join invitation_received ir 
on ra.assessor_id = ir.student_id  where ir.sent_at is not NULL;

delete  from review_associations 
where assessor_id in (select assessor_id 
from (select assessor_id 
from review_associations ra left join invitation_received ir 
on ra.assessor_id = ir.student_id  where ir.sent_at is NULL) x);

# finding students at end of semester after metareview end

In [ ]:
# repo = control_store.skaa_repos[1]
# repo.essay[repo.essay.invited_to_metareview > '2023-12-15']

## Students who have done essay and been assigned a reviewer

In [ ]:
repo.essay

## Students who have not submitted the essay 

In [ ]:
"""
Students who have not submitted the initial work
:return: DataFrame
"""
repo.no_essay

## Students whose reviewer has turned in the review

In [ ]:
"""
Returns the subset of students who have turned in the initial work
whose reviewer has turned in the review
:return: DataFrame
"""        
repo.reviewed
 

## Students with a reviewer who hasn't turned in the review

In [ ]:
   
"""
Returns the subset of students who have turned in the initial work
whose reviewer has NOT turned in the review
"""
# repo.non_reviewed
r = repo.non_reviewed
r[pd.isnull(r.invited_to_metareview)]

## Students whose author has turned in the metareview

In [ ]:
"""
Returns the subset of students who have turned in the initial work
whose author has turned in the metareview
"""
repo.metareviewed

## Students whose author has not turned in the metareview

In [ ]:
"""
Returns the subset of students who have turned in the initial work
whose author has turned in the metareview
"""
repo.non_metareviewed

# Days after official due date

In [ ]:
control_store.skaa_repos[4].data

In [ ]:
pd.to_datetime?

In [ ]:
initial_work_index = 0
review_index = 1
meta_index = 2
due_dates = []

for unit_num in units:
    components = control_store.units[unit_num].components
    due_dates.append({
    'unit' : unit_num,
    'essay' : pd.to_datetime(components[0].due_at,  utc=True),
    'review': pd.to_datetime(components[1].due_at),
    'meta' : pd.to_datetime(components[2].due_at)
    })
due_dates = pd.DataFrame(due_dates)
due_dates.set_index('unit', inplace=True)

In [ ]:
from bokeh.transform import jitter
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6, Plasma, mpl

In [ ]:
# MAX_UNIT = 8

# units = [u for u in range(1,MAX_UNIT + 1)]
subs = []

for u in units:
    try:
        for i, r in control_store.skaa_repos[u].data.iterrows():
            subs.append({ 'unit' : u , 'submitted' : r.invited_to_review} )
    except (KeyError, AttributeError):
        pass

subs = pd.DataFrame(subs)
subs.dropna(inplace=True)
subs

In [ ]:
subs.set_index('unit', inplace=True)

In [ ]:
subs['submitted'] = pd.to_datetime(subs['submitted'], utc=True)

In [ ]:
subs.loc[3]['submitted'].tolist()[0]

nb, using the invited to review will create a few outliers who had to wait for a partner

In [ ]:
after_due = []

for u in units:
    essay_due_date = due_dates.loc[u]['essay']
    print(essay_due_date)

In [ ]:
due_dates.loc[1]['essay']

In [ ]:
    deltas[u] = subs.loc[u]['submitted'] - due_dates.loc[u]['essay']

In [ ]:
deltas = []
for u in units: 
    d = subs.loc[u]['submitted'] - due_dates.loc[u]['essay']
    deltas.append(d)
# deltas = pd.DataFrame(deltas)

deltas = pd.concat(deltas)
deltas = pd.DataFrame(deltas)
deltas.reset_index(inplace=True)
# cast to a decimal value representing number of days
deltas['submitted'] = deltas['submitted'].astype("timedelta64[D]")

In [ ]:
deltas

In [ ]:
sns.violinplot(data=deltas, x='unit', y='submitted')

In [ ]:
sns.boxplot(data=deltas, x='unit', y='submitted')

In [ ]:
source = ColumnDataSource(subs)

mapper = linear_cmap(field_name='unit', palette=mpl['Plasma'][7] ,low=min(units) ,high=max(units))

p = figure(
        plot_width=1000,
        plot_height=500,
        title=f"Essay submissions: Number submitted by day",
        x_axis_type='datetime') #,
p.yaxis.axis_label = "Unit"
p.circle(x='submitted', 
         y=jitter('unit', width=0.6, range=p.y_range),
         legend_label='unit', 
         color=mapper, 
         source=source, 
         alpha=0.9)
p.legend.location = 'top_left'

show(p)

# sources = []
sources = {}

for u in units:
    repo = control_store.skaa_repos[u]
    c = repo.data.invited_to_review.copy(deep=True)
    c = pd.DataFrame(c).dropna().rename({'invited_to_review' : 'date'}, axis=1).set_index('date')
    c['cum_subs'] = 1
    c = c.resample('1D').count()
    c['cum_subs'] = c.cum_subs.cumsum()
    c['unit'] = u
    sources[u] = ColumnDataSource(c.reset_index())
    
#     sources.append((u, ColumnDataSource(c.reset_index())))
#     sources.append(c)

# sources = pd.concat(sources)

def make_color_iter(number=8):
    for m in mpl['Plasma'][number]:
        yield m
    

enrollment = len(control_store.skaa_repos[1].studentRepo.data)

ci = make_color_iter()

TOOLTIPS = [
    ("Total submissions", "@cum_subs{0,0}"),
        ("Unit", "@unit"),
#     ("Date", "@date")
]


p = figure(
        plot_width=1000,
        plot_height=500,
        title=f"Essay submissions: Cumulative submissions",
        x_axis_type='datetime',
tooltips=TOOLTIPS) #,

p.yaxis.axis_label = "Cumulative submissions"


for unit, s in sources.items():
    p.line(
    x='date',
    y='cum_subs',
    line_width=5,
    color=next(ci),
    source=s,
    alpha=0.8,
    legend_label=f"Unit {unit}"
    )

enrolled = Span(location=enrollment,
                dimension='width', line_color='red',
                line_dash='dashed', line_width=3)
p.add_layout(enrolled)
    
p.legend.location = 'bottom_left'

show(p)

# Days after received feedback